# Языковая модель

Построим языковую модель сначала вручную на простом синтетическом корпусе, затем обучим модель из пакета `nltk` на стихотворении "Дом, который построил Джек"

## Импорты

In [1]:
from collections import Counter

import nltk.lm as lm
from nltk.util import ngrams as nltk_ngrams
import numpy as np
import scipy.stats as st

## Пример

In [2]:
text = 'SOS SOS ' + 'А Б ' * 100 + 'EOS'
tokens = text.split()
n = len(tokens)
tokens[:10]

['SOS', 'SOS', 'А', 'Б', 'А', 'Б', 'А', 'Б', 'А', 'Б']

In [3]:
def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

In [4]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(tokens, 3)

In [5]:
ngram_counts

{1: Counter({('SOS',): 2, ('А',): 100, ('Б',): 100, ('EOS',): 1}),
 2: Counter({('SOS', 'SOS'): 1,
          ('SOS', 'А'): 1,
          ('А', 'Б'): 100,
          ('Б', 'А'): 99,
          ('Б', 'EOS'): 1}),
 3: Counter({('SOS', 'SOS', 'А'): 1,
          ('SOS', 'А', 'Б'): 1,
          ('А', 'Б', 'А'): 99,
          ('Б', 'А', 'Б'): 99,
          ('А', 'Б', 'EOS'): 1})}

In [6]:
prefix_counts

{1: Counter({('*',): 203}),
 2: Counter({('SOS', '*'): 2, ('А', '*'): 100, ('Б', '*'): 100}),
 3: Counter({('SOS', 'SOS', '*'): 1,
          ('SOS', 'А', '*'): 1,
          ('А', 'Б', '*'): 100,
          ('Б', 'А', '*'): 99})}

### N-граммы и их частотные вероятности

$\hat p_i = \hat p(w_i)$

In [7]:
def unigram_probas(ngram_counts):
    p1 = {}
    n = sum(ngram_counts[1].values())
    for w in ngram_counts[1]:
        p1[w] = ngram_counts[1][w] / n
    return p1

In [8]:
p1 = unigram_probas(ngram_counts)
p1

{('SOS',): 0.009852216748768473,
 ('А',): 0.49261083743842365,
 ('Б',): 0.49261083743842365,
 ('EOS',): 0.0049261083743842365}

$\hat p_{i, i - 1} = \hat p(w_i|w_{i - 1})$

In [9]:
def bigram_probas(ngram_counts, prefix_counts):
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2

In [10]:
p2 = bigram_probas(ngram_counts, prefix_counts)
p2

{'SOS|SOS': 0.5, 'А|SOS': 0.5, 'Б|А': 1.0, 'А|Б': 0.99, 'EOS|Б': 0.01}

$\hat p_{i, i - 1, i - 2} = \hat p(w_i|w_{i - 1}, w_{i - 2})$

In [11]:
def trigram_probas(ngram_counts, prefix_counts):
    p3 = {}
    for w in ngram_counts[3]:
        pre_w = w[:2] + tuple(['*'])
        p3[u'{2}|{1},{0}'.format(*w)] = ngram_counts[3][w] / prefix_counts[3][pre_w]
    return p3

In [12]:
p3 =  trigram_probas(ngram_counts, prefix_counts)
p3

{'А|SOS,SOS': 1.0,
 'Б|А,SOS': 1.0,
 'А|Б,А': 0.99,
 'Б|А,Б': 1.0,
 'EOS|Б,А': 0.01}

### Проверка гипотезы, что триграммную модель можно свести к биграммной против правосторонней альтернативы

Статистика:

$$-2 \log (\prod_{i, j, k = 1}^m (\hat p_{ij} / \hat p_{ijk})^{n_{ijk}}) = \sum_{i, j, k}^m -2 n_{ijk} \log \hat p_{ij} + 2 n_{ijk} \log \hat p_{ijk} = \sum_{i = 3}^N -2 \log \hat p_{i,i - 1} + 2 \log \hat p_{i, i - 1, i - 2},$$
$$n_{ijk} = |\{X_t: X_t = O_i, X_{t + 1} = O_j, X_{t + 2} = O_k\}|$$

In [13]:
def chi2_statistic(p2, p3, tokens):
    stat2 = []
    stat3 = []
    n = len(tokens)
    for i in range(n - 2):
        w = tokens[i : i + 3]
        ngram3 = '{2}|{1},{0}'.format(*w)
        ngram2 = '{1}|{0}'.format(*w)

        stat2.append(np.log(p2[ngram2]))
        stat3.append(np.log(p3[ngram3]))
    return - 2 * np.sum(stat2) + 2 * np.sum(stat3)

In [14]:
m = len(p3)
stat = chi2_statistic(p2, p3, tokens)

In [15]:
print(f'p-value = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(stat)}')

p-value = 1.0


Гипотеза отвергается

## Другой пример

In [16]:
text = 'SOS SOS ' + 'А Б Б А Б А Б А Б Б А А ' * 100
tokens = text.split()
tokens[:10]

['SOS', 'SOS', 'А', 'Б', 'Б', 'А', 'Б', 'А', 'Б', 'А']

In [17]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(tokens, 3)

In [18]:
ngram_counts

{1: Counter({('SOS',): 2, ('А',): 600, ('Б',): 600}),
 2: Counter({('SOS', 'SOS'): 1,
          ('SOS', 'А'): 1,
          ('А', 'Б'): 400,
          ('Б', 'Б'): 200,
          ('Б', 'А'): 400,
          ('А', 'А'): 199}),
 3: Counter({('SOS', 'SOS', 'А'): 1,
          ('SOS', 'А', 'Б'): 1,
          ('А', 'Б', 'Б'): 200,
          ('Б', 'Б', 'А'): 200,
          ('Б', 'А', 'Б'): 300,
          ('А', 'Б', 'А'): 200,
          ('Б', 'А', 'А'): 100,
          ('А', 'А', 'А'): 99,
          ('А', 'А', 'Б'): 99})}

In [19]:
prefix_counts

{1: Counter({('*',): 1202}),
 2: Counter({('SOS', '*'): 2, ('А', '*'): 599, ('Б', '*'): 600}),
 3: Counter({('SOS', 'SOS', '*'): 1,
          ('SOS', 'А', '*'): 1,
          ('А', 'Б', '*'): 400,
          ('Б', 'Б', '*'): 200,
          ('Б', 'А', '*'): 400,
          ('А', 'А', '*'): 198})}

In [20]:
p1 = unigram_probas(ngram_counts)
p1

{('SOS',): 0.0016638935108153079,
 ('А',): 0.49916805324459235,
 ('Б',): 0.49916805324459235}

In [21]:
p2 = bigram_probas(ngram_counts, prefix_counts)
p2

{'SOS|SOS': 0.5,
 'А|SOS': 0.5,
 'Б|А': 0.667779632721202,
 'Б|Б': 0.3333333333333333,
 'А|Б': 0.6666666666666666,
 'А|А': 0.332220367278798}

In [22]:
p3 =  trigram_probas(ngram_counts, prefix_counts)
p3

{'А|SOS,SOS': 1.0,
 'Б|А,SOS': 1.0,
 'Б|Б,А': 0.5,
 'А|Б,Б': 1.0,
 'Б|А,Б': 0.75,
 'А|Б,А': 0.5,
 'А|А,Б': 0.25,
 'А|А,А': 0.5,
 'Б|А,А': 0.5}

### Проверка той же гипотезы

In [23]:
stat = chi2_statistic(p2, p3, tokens)

In [24]:
print(f'p-value = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(stat)}')

p-value = 0.0


Гипотеза не отвергается

### Сглаживание Лапласа

In [25]:
n1 = list(nltk_ngrams(tokens, 1))
n2 = list(nltk_ngrams(tokens, 2))
n3 = list(nltk_ngrams(tokens, 3))
n3[:10]

[('SOS', 'SOS', 'А'),
 ('SOS', 'А', 'Б'),
 ('А', 'Б', 'Б'),
 ('Б', 'Б', 'А'),
 ('Б', 'А', 'Б'),
 ('А', 'Б', 'А'),
 ('Б', 'А', 'Б'),
 ('А', 'Б', 'А'),
 ('Б', 'А', 'Б'),
 ('А', 'Б', 'Б')]

In [26]:
laplace = lm.Laplace(order=3)
laplace.fit([n1] + [n2] + [n3], vocabulary_text=list(set(tokens)))
regular_lm = lm.MLE(order=3)
regular_lm.fit([n1] + [n2] + [n3], vocabulary_text=list(set(tokens)))

#### Перплексия

(Меньше => лучше)

In [27]:
laplace.perplexity(n1), regular_lm.perplexity(n1)

(2.024429736885131, 2.0224364471218337)

In [28]:
foo = [('b'), ('a'), ('r')]
laplace.perplexity(foo), regular_lm.perplexity(foo)

(1206.000000000001, inf)

#### Сглаженная по Лапласу оценка вероятности

$$p_L(w_i) = \frac{c_i + 1}{\sum_{i = 1}^v c_i + v}$$
$$p_L(w_i|w_j) = \frac{c_{ij} + 1}{\sum_{j=1}^v (c_{ij} + 1)} = \frac{c_{ij} + 1}{c_i + v}$$

$p_L('А'|'SOS')$

In [29]:
laplace.score('А', context=['SOS']), regular_lm.score('А', context=['SOS'])

(0.3333333333333333, 0.5)

$p_L('SOS')$

In [30]:
laplace.score('SOS'), regular_lm.score('SOS')

(0.0024875621890547263, 0.0016638935108153079)

Эти n-граммы не встречались в тексте:

In [31]:
laplace.score('C', context=['SOS']), laplace.score('ыаываа', context=['B']), laplace.score('B')

(0.16666666666666666, 0.25, 0.0008291873963515755)

In [32]:
regular_lm.score('C', context=['SOS']), regular_lm.score('ыаываа', context=['B']), regular_lm.score('B')

(0.0, 0, 0.0)

## Генерация текста

"Дом, который построил Джек"

In [33]:
from nltk.tokenize import RegexpTokenizer
rt = RegexpTokenizer(u'\w+')

In [34]:
with open('jack.txt') as f:
    text = f.read().lower()

In [35]:
tokens = rt.tokenize(text)
len(tokens), len(set(tokens))

(247, 57)

In [36]:
n1 = list(nltk_ngrams(tokens, 1) )
n2 = list(nltk_ngrams(tokens, 2))
n3 = list(nltk_ngrams(tokens, 3))

In [37]:
laplace = lm.Laplace(order=3)
laplace.fit([n1] + [n2] + [n3], vocabulary_text=list(set(tokens)))

In [38]:
' '.join(laplace.generate(50, random_seed=42))

'птица синица которая часто ворует пшеницу которая в тёмном чулане хранится в доме который построил джек а это старушка седая и строгая которая доит корову безрогую лягнувшую старого пса без хвоста который за шиворот треплет кота который пугает и ловит синицу которая часто ворует пшеницу которая в тёмном чулане хранится'

In [39]:
' '.join(laplace.generate(50, text_seed='вот дом который построил джек'.split()))

'вот два петуха которые будят того пастуха который бранится с коровницей строгою которая доит корову безрогую лягнувшую старого пса без хвоста который за шиворот треплет кота который пугает и ловит синицу которая часто ворует пшеницу которая в тёмном чулане хранится в доме который построил джек а это корова безрогая лягнувшая'

In [40]:
' '.join(laplace.generate(50, text_seed='привет как дела'.split()))

'два петуха которые будят того пастуха который бранится с коровницей строгою которая доит корову безрогую лягнувшую старого пса без хвоста который за шиворот треплет кота который пугает и ловит синицу которая часто ворует пшеницу которая в тёмном чулане хранится в доме который построил джек а это весёлая птица синица которая'